## Bike renting dataset
El conjunto de datos de alquiler de bicicletas por dias incluye registros detallados del uso compartido de bicicletas en una ciudad, con cada registro representando un dia específico. Las características del dataset son:



<ul>
<li>dteday: Marca temporal que indica la fecha.
<li>season: Estación del año (1: invierno, 2: primavera, 3: verano, 4: otoño).
<li>yr: Año de la observación (por ejemplo, 0: 2011, 1: 2012).
<li>mnth: Mes del año (1 a 12).
<li>holiday: Indicador de si el día es festivo (0: no festivo, 1: festivo).
<li>weekday: Día de la semana (0: domingo, 1: lunes, ..., 6: sábado).
<li>workingday: Indicador de si es un día laboral (0: fin de semana o festivo, 1: día laboral).
<li>weathersit: Estado del clima (1: despejado, 2: nublado, 3: lluvia ligera, 4: lluvia fuerte).
<li>temp: Temperatura normalizada en grados Celsius.
<li>atemp: Sensación térmica normalizada en grados Celsius.
<li>hum: Humedad relativa normalizada.
<li>windspeed: Velocidad del viento normalizada.
<li>casual: Número de usuarios casuales que alquilan bicicletas sin suscripción.
<li>registered: Número de usuarios registrados que alquilan bicicletas con suscripción.
<li>cnt: Número total de bicicletas alquiladas en esa hora (suma de usuarios casuales y registrados).
</ul>

Se puede descargar el conjunto de datos en https://www.kaggle.com/datasets/lakshmi25npathi/bike-sharing-dataset

In [ ]:
import sys
import pandas as pd
from pandas import read_csv
import numpy
from numpy import array
import matplotlib.pyplot as plt
import math
import tensorflow
import keras
import sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from google.colab import drive
import os
from contextlib import nullcontext


# Variables iniciales
Historia=1
BATCHSIZE=1
EPOCAS=1
LSTM_DIM=1


In [ ]:
print (sys.version_info)
print ("NumPy: ", numpy.__version__)
print ("Pandas: ", pd.__version__)
print ("TensorFlow: ", tensorflow.__version__)
print ("Keras: ", keras.__version__)
print ("SciKitLearn: ", sklearn.__version__)


<h1>Carga del fichero</h1>

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/proyecto-tensorflow/buenos/day.csv'
dfBikes=pd.read_csv (path)

In [ ]:
# Ver nombres de la columnas
dfBikes.columns

In [ ]:
# Renombramos las columnas
dfBikes.rename(columns = {'instant':'Id', 'dteday':'Fecha',
                     "season":'Estacion', 'yr':'Anio', 'mnth':'Mes',
                     'holiday':'Festivo','weekday':'DiaSemana','workingday':'DiaTrabajo','weathersit':'Tiempo','Temperatura':'temp','SensacionTemperatura':'atemp','Humedad':'hum','windspeed':'VelocidadViento','casual':'Casual','registered':'Registrado','cnt':'Cuenta'}, inplace=True)

# Ordenamos por el id
dfBikes.sort_values(['Id'], inplace=True)


In [ ]:
# Contar el número de valores nulos por columna
valores_nulos_por_columna = dfBikes.isnull().sum()

# Mostrar el resultado
print(valores_nulos_por_columna)

In [ ]:
# No es necesario porque no hay nulos
dfBikes.dropna(inplace=True)
dfBikes

## Funciones

In [ ]:
# Convertir los datos en un formato adecuado para entrenar modelos de series temporales, en este caso LSTM
def CreaDatos(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


# Función para visualizar en un gráfico la predicción y el valor real
# Aquí se desnormalizan los datos. Para ello, se despeja la fórmula de normalización
def CalculoRMSE (titulo2):
    # En base al modelo establecido, se realizan predicciones y se desnormalizan los datos
    testPredict = model.predict(testX, batch_size=BATCHSIZE)*(MaxRealY - MinRealY) + MinRealY
    # Los valores reales se desnormalizan
    testReal=testY[:,13]*(MaxRealY - MinRealY) + MinRealY

    # Se calcula el RMSe
    testScore = math.sqrt(mean_squared_error(testReal, testPredict.reshape(-1,1)))

    # Se realiza el gráfico
    plt.plot (testPredict, label="Predict")
    plt.plot (testReal, label="Real")
    plt.legend ()
    plt.title ("Prediccion de bicicletas alquiladas en el dia.\n"+titulo2+" Historia="+str (Historia)+" Epocas="+str (EPOCAS)+ " LSTM_dim="+str (LSTM_DIM))
    plt.show()
    return testScore


<h1>Procesamiento de los datos</h1>

In [ ]:
# Seleccionamos las columnas a utilizar. Todas menos la de la fecha y la del id
df = dfBikes.iloc[:,2:]
print(df)
print(df.shape)

In [ ]:
# Buscamos el mayor y menor valor de la clase
MinRealY=df["Cuenta"].min()
MaxRealY=df["Cuenta"].max()
print(MinRealY)
print(MaxRealY)

In [ ]:
# Normalizamos los datos
scaler  = MinMaxScaler()
dataset = scaler.fit_transform(df)
print(dataset)

In [ ]:
# Separamos en conjunto de entrenamiento y de test
train_size = int(len(df) * 0.67)
test_size  = len(df) - train_size
train = dataset[0:train_size,:]
test  = dataset[train_size:len(df),:]

In [ ]:
test

In [ ]:
# Como se desnormaliza
desnomalizar=test[:,13]*(MaxRealY - MinRealY) + MinRealY
desnomalizar

<h1>MLP</h1>

In [ ]:
from tensorflow.keras import datasets, layers, models

In [ ]:
# Preparar los datos separando la clase de los atributos utilizados para realizar las predicciones
trainX=train[:,0:12]
trainY=train[:,13]
testX=test[:,0:12]
testY=test[:,13]


In [ ]:

# Dataframe para almacenar las combinaciones de parametros y su RMSE medio obtenido en las 3 iteraciones
dfError = pd.DataFrame(columns=["opt","capas","perdida","batch","epoca", "RMSE"])

# Parámetros a evaluar y sus valores diferentes
optimizadores=["SGD","adam","RMSprop"]
funciones_perdidas=["mean_squared_error","mean_absolute_error","huber_loss"]
batch_sizes=[16,32,64]
epocas=[6,10,14]
capas=[1,3,5]

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
trainX_reduced=trainX[:50]
trainY_reduced=trainY[:50]
testX_reduced=testX[:20]
testY_reduced=testY[:20]

In [ ]:
print(trainX_reduced.shape)
print(trainY_reduced.shape)
print(testX_reduced.shape)
print(testY_reduced.shape)

In [ ]:

# Recorrer parámetros
for capas_ind in range(len(capas)):
  for perdida_ind in range(len(funciones_perdidas)):
    for batch_ind in range(len(batch_sizes)):
      for epoca_ind in range(len(epocas)):
        for opt_ind in range(len(optimizadores)):

          # Imprimir parámetros que se evaluan
          print("\n\n\n")
          print("capas:",str(capas_ind))
          print("perdida:",str(perdida_ind))
          print("batch:",str(batch_ind))
          print("epoca:",str(epoca_ind))
          print("opt:",str(opt_ind))

          # Se establece el optimizador
          optimizer=None
          if(optimizadores[opt_ind]=="SGD"):
            optimizer = keras.optimizers.SGD(learning_rate=1e-3)
            print("SGD")
          if(optimizadores[opt_ind]=="adam"):
            optimizer = keras.optimizers.Adam(learning_rate=1e-3)
            print("Adam")
          if(optimizadores[opt_ind]=="RMSprop"):
            optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
            print("RMSprop")

          # Se establece el modelo
          modelNN_Keras = models.Sequential()
          # Aquí input_dim es trainX.shape[1] que es 12 porque existen 12 atributos de entrada
          # Esta capa de entrada se mantiene siempre
          modelNN_Keras.add(layers.Dense(units = 16, activation = 'relu', input_dim = trainX.shape[1]))

          # Después añadimos las capas intermedias que correspondan segun el valor del parámetro
          for _ in range(capas[capas_ind]):
            modelNN_Keras.add(layers.Dense(units = 6, activation = 'relu'))

          # La capa de salida se mantiene siempre y tiene como función de activación sigmoid
          modelNN_Keras.add(layers.Dense(units = 1, activation = 'sigmoid'))
          modelNN_Keras.summary()

          # Para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteración
          i=0
          acumulador=0

          # Repetimos 3 veces para cada modelo con su combinación de parámetros el proceso de entrenamiento y predicción
          for i in range(3):
              # Se ajusta el optimizador y la función de pérdida
              modelNN_Keras.compile(optimizer=optimizer, loss=funciones_perdidas[perdida_ind], metrics=['accuracy'])
              # Se ajustan las épocas y el batch_size
              history = modelNN_Keras.fit(trainX_reduced, trainY_reduced, validation_split = 0.1, epochs=epocas[epoca_ind], batch_size=batch_sizes[batch_ind], verbose=0)
              # Se realiza predicción
              y_pred = modelNN_Keras.predict(testX_reduced)
              # Evalua RMSE y acumula su valor
              rmse = math.sqrt(mean_squared_error(testY_reduced, y_pred))
              acumulador=acumulador+rmse
              print("Ejecucción: ",str(i))

          # Calcula la media de RMSE en las 3 iteraciones
          media=acumulador/3
          print("Media: ",str(media))
          # Se almacena en el dataframe junto a los parámetros seleccionados
          dfError.loc[len(dfError)] = [str(optimizadores[opt_ind]),str(capas[capas_ind]),str(funciones_perdidas[perdida_ind]),str(batch_sizes[batch_ind]),str(epocas[epoca_ind]), media]
          print("Completado parámetros\n\n")


print(dfError)
dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/proyecto-tensorflow/buenos/MLP_resultados.csv", index=False)




<h1>Preparación de los datos para LSTM</h1>

In [ ]:
# Convertimos los datos en un formato adecuado para LSTM
trainX, trainY = CreaDatos(train, Historia)
testX, testY   = CreaDatos(test, Historia)

In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:
trainX_reduced=trainX[:50]
trainY_reduced=trainY[:50]
testX_reduced=testX[:20]
testY_reduced=testY[:20]

In [ ]:
print(trainX_reduced.shape)
print(trainY_reduced.shape)
print(testX_reduced.shape)
print(testY_reduced.shape)

<h1>LSTM para un problema de regresion</h1>

In [ ]:

# Dataframe para almacenar el RMSE con las diferentes combinaciones de parámetros
dfError = pd.DataFrame(columns=["func_salida","LSTM_dim","batch","epoca", "RMSE"])

# Parámetros a evaluar y sus valores diferentes
func_salida=["sigmoid","relu","tanh","linear"]
batch_sizes=[16,32,64]
epocas=[6,10,14]
LSTM_dim=[4,8,12,16]

func_salida_ind=0
batch_ind=0
epocas_ind=0
LSTM_dim_ind=0

# Recorrer parámetros
for func_salida_ind in range(len(func_salida)):
  for batch_ind in range(len(batch_sizes)):
    for epocas_ind in range(len(epocas)):
      for LSTM_dim_ind in range(len(LSTM_dim)):

        # Imprimir valores valuados
        print("\n\n\n")
        print("func_salida:",str(func_salida_ind))
        print("LSTM_dim:",str(LSTM_dim_ind))
        print("batch:",str(batch_ind))
        print("epoca:",str(epocas_ind))
        # Ajustar los parámetros para cuando vaya a hacer la predicción en la función CalculoRMSE
        BATCHSIZE=batch_sizes[batch_ind]
        EPOCAS=epocas[epocas_ind]
        LSTM_DIM=LSTM_dim[LSTM_dim_ind]

        # Crear el modelo
        model = Sequential()
        # Ajustar parámetro lstm_dim
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], input_shape=(trainX.shape[1],trainX.shape[2])))
        # Ajustar parámetro función de salida
        model.add(Dense(1, activation=func_salida[func_salida_ind]))
        model.compile(loss='mean_squared_error', optimizer='adam')
        model.summary()

        # Para realizar las 3 iteraciones y acumular el RMSE de cada iteración
        i=0
        acumulador=0

        # Realizar 3 veces con el modelo el entrenamiento
        for i in range(3):

            # Ajustar el batch_size y el número de épocas
            model.fit(trainX, trainY[:,13], epochs=epocas[epocas_ind], batch_size=batch_sizes[batch_ind], verbose=0)

            # Se llama a la función que obtiene el RMSE y realiza la gráfica para comparar los valores reales con los predichos
            testScore=CalculoRMSE ("LSTM-vainilla")
            acumulador=acumulador+testScore
            print("Ejecucción: ",str(i))

        # Calcular la media de RMSE en las 3 iteraciones
        media=acumulador/3
        print("media: ",str(media))
        # Se almacena en el dataframe junto a los parámetros seleccionados
        dfError.loc[len(dfError)] = [str(func_salida[func_salida_ind]),str(LSTM_dim[LSTM_dim_ind]),str(batch_sizes[batch_ind]),str(epocas[epocas_ind]), media]
        print("Completado parámetros\n\n")

# Guardar el dataframe
print(dfError)
dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/proyecto-tensorflow/buenos/LSTM_1_resultados.csv", index=False)




<h1>LSTM utilizando una ventana temporal</h1>

In [ ]:
# Convertir los datos en un formato adecuado para LSTM
Historia = 3
trainX, trainY = CreaDatos (train, Historia)
testX, testY   = CreaDatos (test, Historia)



In [ ]:
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)

In [ ]:

# Dataframe para almacenar las combinaciones de parámetros y su RMSE medio obtenido en las 3 iteraciones
dfError = pd.DataFrame(columns=["opt","LSTM_dim","perdida","epoca", "RMSE"])

# Parámetros a evaluar y sus valores diferentes
optimizadores=["SGD","adam","RMSprop"]
funciones_perdidas=["mean_squared_error","mean_absolute_error","huber_loss"]
epocas=[6,10,14]
LSTM_dim=[2,5,10]



LSTM_dim_ind=0
perdida_ind=0
epoca_ind=0
opt_ind=0

# Recorrer parametros
for LSTM_dim_ind in range(len(LSTM_dim)):
  for perdida_ind in range(len(funciones_perdidas)):
    for epoca_ind in range(len(epocas)):
        for opt_ind in range(len(optimizadores)):

          # Imprimir por donde llega
          print("\n\n\n")
          print("LSTM_dim:",str(LSTM_dim_ind))
          print("perdida:",str(perdida_ind))
          print("epoca:",str(epoca_ind))
          print("opt:",str(opt_ind))
          # Ajustar los parámetros para cuando vaya a hacer la predicción en la función CalculoRMSE
          BATCHSIZE=1
          EPOCAS=epocas[epoca_ind]
          LSTM_DIM=LSTM_dim[LSTM_dim_ind]

          # Establecer el optimizador
          optimizer=None
          if(optimizadores[opt_ind]=="SGD"):
            optimizer = keras.optimizers.SGD(learning_rate=1e-3)
            print("SGD")
          if(optimizadores[opt_ind]=="adam"):
            optimizer = keras.optimizers.Adam(learning_rate=1e-3)
            print("Adam")
          if(optimizadores[opt_ind]=="RMSprop"):
            optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
            print("RMSprop")

          # Crear el modelo
          model = Sequential()
          # Ajustar parámetro LSTM_dim
          model.add(LSTM(LSTM_dim[LSTM_dim_ind], input_shape=(trainX.shape [1], trainX.shape[2])))
          model.add(Dense(1, activation='sigmoid'))
          # Ajustar optimizador y función de perdida
          model.compile(loss=funciones_perdidas[perdida_ind], optimizer=optimizer)

          # Para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteración
          i=0
          acumulador=0

          for i in range(3):
              # Entrenar el modelo y ajustar épocas
              model.fit(trainX, trainY[:,13], epochs=epocas[epoca_ind], batch_size=1, verbose=0)

              # llamar a la función que obtiene el RMSE y realiza la gráfica para comparar los valores reales con los predichos
              testScore=CalculoRMSE ("LSTM-vainilla")
              acumulador=acumulador+testScore
              print("ejecuccion: ",str(i))

          # Calcular la media de RMSE en las 3 iteraciones
          media=acumulador/3
          print("media: ",str(media))
          # Almacenarlo en el dataframe junto a los parámetros seleccionados
          dfError.loc[len(dfError)] = [str(optimizadores[opt_ind]),str(LSTM_dim[LSTM_dim_ind]),str(funciones_perdidas[perdida_ind]),str(epocas[epoca_ind]), media]
          print([str(optimizadores[opt_ind]),str(LSTM_dim[LSTM_dim_ind]),str(funciones_perdidas[perdida_ind]),str(epocas[epoca_ind]), media])
          print("Completado parámetros\n\n")

          # Guardar el dataframe
          print(dfError)
          dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/proyecto-tensorflow/buenos/LSTM_2_resultados.csv", index=False)




<h1>LSTM apiladas</h1>

In [ ]:
# Convertir los datos en un formato adecuado para LSTM
historia = 1
trainX, trainY = CreaDatos (train, historia)
testX, testY   = CreaDatos (test, historia)

print (trainX.shape)

In [ ]:
# Dataframe para almacenar el RMSE con las diferentes combinaciones de parámetros
dfError = pd.DataFrame(columns=["opt","batch","epoca","LSTM_dim", "RMSE"])

# Parámetros a evaluar y sus valores diferentes
optimizadores=["RMSprop","adam","SGD"]
batch_sizes=[16,32,64]
epocas=[6,10,14]
LSTM_dim=[4,8,12]


func_perdida_ind=0
batch_ind=0
epocas_ind=0
LSTM_dim_ind=0
opt_ind=0

# Recorrer parámetros
for batch_ind in range(len(batch_sizes)):
  for epocas_ind in range(len(epocas)):
    for LSTM_dim_ind in range(len(LSTM_dim)):
      for opt_ind in range(len(optimizadores)):

        # Imprimir por donde llega
        print("\n\n\n")
        print("LSTM_dim:",str(LSTM_dim_ind))
        print("batch:",str(batch_ind))
        print("epoca:",str(epocas_ind))
        print("optimizador:",str(opt_ind))
        # Ajustar los parámetros para cuando vaya a hacer la predicción en la función CalculoRMSE
        BATCHSIZE=batch_sizes[batch_ind]
        EPOCAS=epocas[epocas_ind]
        LSTM_DIM=LSTM_dim[LSTM_dim_ind]


        # Establecer el optimizador
        optimizer=None
        if(optimizadores[opt_ind]=="SGD"):
          optimizer = keras.optimizers.SGD(learning_rate=1e-3)
          print("SGD")
        if(optimizadores[opt_ind]=="adam"):
          optimizer = keras.optimizers.Adam(learning_rate=1e-3)
          print("Adam")
        if(optimizadores[opt_ind]=="RMSprop"):
          optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
          print("RMSprop")



        # Crear el modelo y entrenar
        model = Sequential()
        # Ajustar parámetro LSTM_Dim
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], input_shape=(trainX.shape[1],trainX.shape[2]), return_sequences=True))
        model.add(LSTM(LSTM_dim[LSTM_dim_ind]))
        model.add(Dense(1, activation='sigmoid'))
        # Ajustar optimizador y función de pérdida
        model.compile(loss="mean_squared_error", optimizer=optimizer)


        # Para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteración
        i=0
        acumulador=0

        # Entrenar modelo 3 veces
        for i in range(3):
            # Ajustar épocas y batch size
            model.fit(trainX, trainY[:,13], epochs=epocas[epocas_ind], batch_size=batch_sizes[batch_ind], verbose=0)
            # Llamar a la función que obtiene el RMSE y realiza la gráfica para comparar los valores reales con los predichos
            testScore=CalculoRMSE ("LSTM-dos capas")
            acumulador=acumulador+testScore
            print("ejecuccion: ",str(i))

        # Calcular la media de RMSE en las 3 iteraciones
        media=acumulador/3
        print("media: ",str(media))
        # Lo almacena en el dataframe junto a los parámetros seleccionados
        dfError.loc[len(dfError)] = [str(optimizadores[opt_ind]),str(batch_sizes[batch_ind]),str(epocas[epocas_ind]),str(LSTM_dim[LSTM_dim_ind]), media]
        print("completado parametros\n\n")

        # Guardar el dataframe
        print(dfError)
        dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/proyecto-tensorflow/buenos/LSTM_3_resultados.csv", index=False)




<h1>LSTM con memoria entre lote y lote</h1>

In [ ]:
# Dataframe para almacenar el RMSE con las diferentes combinaciones de parámetros
dfError = pd.DataFrame(columns=["func_act","opt","epoca","LSTM_dim", "RMSE"])

# Parámetros a evaluar y sus valores diferentes
func_activacion=["sigmoid","relu","tanh","linear"]
optimizadores=["RMSprop","SGD","adam"]
epocas=[6,10,14]
LSTM_dim=[4,8,12]



# Recorrer parámetros
for func_activacion_ind in range(len(func_activacion)):
  for opt_ind in range(len(optimizadores)):
    for epocas_ind in range(len(epocas)):
      for LSTM_dim_ind in range(len(LSTM_dim)):

        # Imprimir por donde llega
        print("\n\n\n")
        print("func_acticacion:",str(func_activacion_ind))
        print("LSTM_dim:",str(LSTM_dim_ind))
        print("optimizador:",str(opt_ind))
        print("epoca:",str(epocas_ind))
        # Ajustar los parámetros para cuando vaya a hacer la predicción en la función CalculoRMSE
        BATCHSIZE=1
        EPOCAS=epocas[epocas_ind]
        LSTM_DIM=LSTM_dim[LSTM_dim_ind]

        # Establecer el optimizador
        optimizer=None
        if(optimizadores[opt_ind]=="SGD"):
          optimizer = keras.optimizers.SGD(learning_rate=1e-3)
          print("SGD")
        if(optimizadores[opt_ind]=="adam"):
          optimizer = keras.optimizers.Adam(learning_rate=1e-3)
          print("Adam")
        if(optimizadores[opt_ind]=="RMSprop"):
          optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
          print("RMSprop")



        # Crear el modelo y entrenar
        batch_size_alterado = 1
        model = Sequential()
        # Ajustar parámetro LSTM_dim
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], batch_input_shape=(batch_size_alterado, trainX.shape [1], trainX.shape [2]), stateful=True))
        # Después se añaden las capas intermedias que correspondan segun el valor del parámetro

        # Capa de salida se mantiene
        model.add(Dense(1, activation=func_activacion[func_activacion_ind]))
        # Ajustar optimizador
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        # Para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteración
        i=0
        acumulador=0
        k=0

        # Repetir 3 veces el entrenamiento
        for i in range(3):
            # En este modelo, se realizan tantas iteraciones como épocas
            for k in range(epocas[epocas_ind]):
                # Ajustar el parámetro de batch
                model.fit(trainX, trainY[:,13], epochs=1, batch_size=1, verbose=0, shuffle=False)

                # Llamar a la función que obtiene el RMSE y realiza la gráfica para comparar los valores reales con los predichos
                testScore=CalculoRMSE("LSTM - con memoria entre lote y lote")
                acumulador=acumulador+testScore
                print("ejecuccion: ",str(i))

                model.reset_states()


        # Calcular la media de RMSE en las 3 iteraciones
        media=acumulador/3
        print("media: ",str(media))
        #Lo almacena en el dataframe junto a los parámetros seleccionados
        dfError.loc[len(dfError)] = [str(func_activacion[func_activacion_ind]),str(optimizadores[opt_ind]),str(epocas[epocas_ind]),str(LSTM_dim[LSTM_dim_ind]), media]
        print("Completado parámetros\n\n")

        # Guardar el dataframe
        print(dfError)
        dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/proyecto-tensorflow/buenos/LSTM_4_resultados.csv", index=False)




<h1>LSTM's apilados (varias capas) con memoria entre lote y lote</h1>

In [ ]:

# Dataframe para almacenar el RMSE con las diferentes combinaciones de parámetros
dfError = pd.DataFrame(columns=["func_activacion","opt","epoca","LSTM_dim", "RMSE"])

# Parámetros a evaluar y sus valores diferentes
func_activacion=["sigmoid","relu","tanh","linear"]
optimizadores=["RMSprop","SGD","adam"]
epocas=[6,10,14]
LSTM_dim=[4,8,12,16]



# Recorrer parámetros
for func_activacion_ind in range(len(func_activacion)):
  for opt_ind in range(len(optimizadores)):
    for epocas_ind in range(len(epocas)):
      for LSTM_dim_ind in range(len(LSTM_dim)):

        # Imprimir por donde llega
        print("\n\n\n")
        print("func_activacion:",str(func_activacion_ind))
        print("LSTM_dim:",str(LSTM_dim_ind))
        print("optimizador:",str(opt_ind))
        print("epoca:",str(epocas_ind))
        # Ajustar los parámetros para cuando vaya a hacer la predicción en la función CalculoRMSE
        BATCHSIZE=1
        EPOCAS=epocas[epocas_ind]
        LSTM_DIM=LSTM_dim[LSTM_dim_ind]

        # Establecer el optimizador
        optimizer=None
        if(optimizadores[opt_ind]=="SGD"):
          optimizer = keras.optimizers.SGD(learning_rate=1e-3)
          print("SGD")
        if(optimizadores[opt_ind]=="adam"):
          optimizer = keras.optimizers.Adam(learning_rate=1e-3)
          print("Adam")
        if(optimizadores[opt_ind]=="RMSprop"):
          optimizer = keras.optimizers.RMSprop(learning_rate=1e-3)
          print("RMSprop")

        # Crear el modelo y entrenar
        batch_size_alterado = 1
        model = Sequential()
        # Ajustar parámetro LSTM_dim
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], batch_input_shape=(batch_size_alterado, trainX.shape[1], trainX.shape[2]), stateful=True, return_sequences=True))
        model.add(LSTM(LSTM_dim[LSTM_dim_ind], batch_input_shape=(batch_size_alterado, trainX.shape[1], trainX.shape[2]), stateful=True))
        # Ajustar parámetro función activación salida
        model.add(Dense(1, activation=func_activacion[func_activacion_ind]))
        model.compile(loss='mean_squared_error', optimizer=optimizer)

        # Para repetir 3 veces el entreno sobre cada modelo y acumular el RMSE de cada iteración
        i=0
        acumulador=0
        k=0

        # Repetir 3 veces el entreno
        for i in range(3):
            # Recorrer bucle tantas épocas se tengan
            for k in range(epocas[epocas_ind]):
                # Ajustar parámetro batch_size
                model.fit(trainX, trainY[:,13], epochs=1, batch_size=BATCHSIZE, verbose=0, shuffle=False)
                # Llamar a la función que obtiene el RMSE y realiza la gráfica para comparar los valores reales con los predichos
                testScore=CalculoRMSE("prueba")
                acumulador=acumulador+testScore
                print("ejecuccion: ",str(i))

                model.reset_states()


        # Calcular la media de RMSE en las 3 iteraciones
        media=acumulador/3
        print("media: ",str(media))
        # Lo almacena en el dataframe junto a los parámetros seleccionados
        dfError.loc[len(dfError)] = [str(func_activacion[func_activacion_ind]),str(optimizadores[opt_ind]),str(epocas[epocas_ind]),str(LSTM_dim[LSTM_dim_ind]), media]
        print("Completado parámetros\n\n")

        # Guardar el dataframe
        print(dfError)
        dfError.to_csv("/content/drive/MyDrive/Colab Notebooks/proyecto-tensorflow/buenos/LSTM_5_resultados.csv", index=False)


